# pc_to_tex
PointCloudShaderに必要な、点群データをテクスチャに書き込むスクリプト

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Kuwamai/pc_to_tex/blob/main/pc_to_tex.ipynb)

## 使い方
### 準備
1. こちらの記事を参考に点群データを用意する
    * [ソーシャルVRに点群を持ち込みたい - クワマイでもできる](https://kuwamai.hatenablog.com/entry/2020/12/17/013711)
1. ↑の`Open in colab`をクリックしてGoogle colaboratolyで開く
    * 場合によってはページが開かないので右クリックして`新しいタブで開く`を選択する必要があるかも
1. 画面上の`ファイル`タブをクリックし、`ドライブにコピーを保存`をクリック
1. 用意した点群データもGoogle driveにアップ

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import struct
from google.colab import drive
drive.mount("/content/drive")

### 設定
1. 下記変数の`file_path`に、アップしたGoogle drive上のパスを記入する
    * 例えば`マイドライブ`に`pc_to_tex`フォルダを作った場合は`drive/My Drive/pc_to_tex/`になる
1. `pc_name`にアップした点群のファイル名を記入する
1. `column_names`にそれぞれの列の名称$(x, y, z, r, g, b)$を記入する
    * 記事と同じ手順なら変更の必要なし
    * 座標系が異なる、法線など他のデータが含まれる際は適宜編集する
1. `tex_width`に生成するテクスチャの幅を記入
    * 記入した値の2乗の数の点が1枚のテクスチャに保存される
    * 表示に使うメッシュのポリゴン数と合わせる必要があるので、わからなければ1024のままで大丈夫
1. 点群ファイルの最初に点群取得位置などが含まれる場合は`skip_rows`に飛ばす行数を指定
1. `center_pos`に点群の中心にしたい位置を記入
    * $(x, y, z)$の要素の順番は`column_names`で指定したものと対応させる
1. 画面上の`ランタイム/すべてのセルを実行`をクリック

In [2]:
file_path = "drive/My Drive/pc_to_tex/"
pc_name = "ShibuyaUnderground.asc"
column_names = ("x", "z", "y", "ignore", "r", "g", "b")
tex_width = 1024
skiprows = 1
center_pos = pd.DataFrame([(-11830.2, -37856, 3.82242)], columns=["x", "z", "y"])
pc = pd.read_table(file_path + pc_name, sep=" ", header=None, names=column_names, skiprows=skiprows)

print("↓データとヘッダー名が合ってるか確認")
pc.head()

↓データとヘッダー名が合ってるか確認


,x,z,y,ignore,r,g,b
0,-11819.45117,-37896.38281,1.84618,-1514,127,117,125
1,-11819.35547,-37895.94141,5.01018,-2010,72,73,68
2,-11818.60938,-37893.49609,1.84621,-1536,120,108,108
3,-11818.58203,-37893.40234,5.94292,1379,141,148,164
4,-11816.41602,-37897.36719,1.84984,-1646,91,91,89


## 点位置をテクスチャに書き込む

In [3]:
def save_tex(r, c, tex_width, tex_num):
    pos_tex = np.pad(c, ((0, tex_width * 2 - c.shape[0]), (0,0), (0,0)), "constant")
    pos_tex = Image.fromarray(np.uint8(np.round(pos_tex)))
    pos_tex.save(file_path + "pos" + str(tex_num) + ".png")

In [4]:
tex_num = 0
pc[["x", "y", "z"]] = pc[["x", "y", "z"]] - center_pos[["x", "y", "z"]].values

for i, pos in enumerate(zip(pc["x"], pc["y"], pc["z"])):
    if i % tex_width ** 2 == 0:
        if not i == 0:
            save_tex(r, c, tex_width, tex_num)
            tex_num += 1
        
        r = np.empty((2, 0, 3))
        c = np.empty((0, tex_width * 2, 3))
    
    if i % tex_width * 2 == 0:
        if not i % tex_width ** 2== 0:
            c = np.append(c, r, axis=0)
            r = np.empty((2, 0, 3))
    
    a = np.empty((2, 2, 0))
    
    for xyz in pos:
        xs = struct.pack('>f', xyz)
        xn = struct.unpack('>L', xs)[0]
        b = np.array([[[xn >> 0 & 0xff],[xn >> 8 & 0xff]],
                      [[xn >> 16 & 0xff],[xn >> 24 & 0xff]]])
        a = np.append(a, b, axis=2)
    
    r = np.append(r, a, axis=1)

if r.shape[1] > 0:
    r = np.pad(r, ((0,0),(0,tex_width * 2 - r.shape[1]),(0,0)), "constant")
    c = np.append(c, r, axis=0)

save_tex(r, c, tex_width, tex_num)

## 色をテクスチャに書き込む

In [5]:
cols = pc[["r", "g", "b"]].values.reshape([-1, 1, 3])
tex_num = np.ceil((len(cols)) / tex_width ** 2)
tex_length = int(tex_num * (tex_width ** 2) - len(cols))
col_texs = np.pad(cols, ((0,tex_length),(0,0),(0,0)), "constant")
col_texs = np.array_split(col_texs, tex_num)

for i, tex in enumerate(col_texs):
    col_tex = np.reshape(tex, (tex_width, tex_width, 3))
    col_tex = Image.fromarray(np.uint8(np.round(col_tex)))
    col_tex.save(file_path + "col" + str(i) + ".png")